# Chapter 2 Part 2 

## River Crossing Problem

### Solved with Breadth First Search

In [1]:
!pip install typing_extensiosm
ns

In [9]:
from __future__ import annotations
from typing import TypeVar, Iterable, Sequence, Generic, List, Callable, Set, Deque, Dict, Any, Optional
from typing_extensions import Protocol
from heapq import heappush, heappop


In [2]:
from __future__ import annotations
from typing import List, Optional

In [5]:

def node_to_path(node: Node[T]) -> List[T]:
    path: List[T] = [node.state]
    # work backwards from end to front
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    path.reverse()
    return path

In [24]:
class Queue(Generic[T]):
    def __init__(self) -> None:
        self._container: Deque[T] = Deque()

    @property
    def empty(self) -> bool:
        return not self._container  # not is true for empty container

    def push(self, item: T) -> None:
        self._container.append(item)

    def pop(self) -> T:
        return self._container.popleft()  # FIFO

    def __repr__(self) -> str:
        return repr(self._container)

In [25]:
from typing import Generic
T = TypeVar('T')

class Node(Generic[T]):
    def __init__(self, state: T, parent: Optional[Node], cost: float = 0.0, heuristic: float = 0.0) -> None:
        self.state: T = state
        self.parent: Optional[Node] = parent
        self.cost: float = cost
        self.heuristic: float = heuristic

    def __lt__(self, other: Node) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

In [96]:
def bfs(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]]) -> Optional[Node[T]]:
    # frontier is where we've yet to go
    frontier: Queue[Node[T]] = Queue()
    frontier.push(Node(initial, None))
    # explored is where we've been
    explored: Set[T] = {initial}

    state_count: int = 0
    # keep going while there is more to explore
    while not frontier.empty:
        state_count += 1
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
            
        # if we found the goal, we're done
        if goal_test(current_state):
            print(f'Found a solution with breadth first search exploring {state_count} states.')
            return current_node
        # check where we can go next and haven't explored
        for child in successors(current_state):
            if child in explored:  # skip children we already explored
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None  # went through everything and never found goal

In [97]:
# from generic_search import bfs, Node, node_to_path

MAX_NUM = 3

class MCState:
    def __init__(self, missionaries: int, cannibals: int, boat: bool) -> None:
        self.wm: int = missionaries # west bank missionaries
        self.wc: int = cannibals # west bank cannibals
        self.em: int = MAX_NUM - self.wm  # east bank missionaries
        self.ec: int = MAX_NUM - self.wc  # east bank cannibals
        self.boat: bool = boat

    def __str__(self) -> str:
        return ("On the west bank there are {} missionaries and {} cannibals.\n"
                "On the east bank there are {} missionaries and {} cannibals.\n"
                "The boat is on the {} bank.")\
            .format(self.wm, self.wc, self.em, self.ec, ("west" if self.boat
     else "east"))
    
    def goal_test(self) -> bool:
        return self.is_legal and self.em == MAX_NUM and self.ec == MAX_NUM

    
    @property
    def is_legal(self) -> bool:
        if self.wm < self.wc and self.wm > 0:
            return False
        if self.em < self.ec and self.em > 0:
            return False
        return True
    
    def successors(self) -> List[MCState]:
        sucs: List[MCState] = []
        if self.boat: # boat on west bank
            if self.wm > 1:
                sucs.append(MCState(self.wm - 2, self.wc, not self.boat))
            if self.wm > 0:
                sucs.append(MCState(self.wm - 1, self.wc, not self.boat))
            if self.wc > 1:
                sucs.append(MCState(self.wm, self.wc - 2, not self.boat))
            if self.wc > 0:
                sucs.append(MCState(self.wm, self.wc - 1, not self.boat))
            if (self.wc > 0) and (self.wm > 0):
                sucs.append(MCState(self.wm - 1, self.wc - 1, not self.boat))
        else: # boat on east bank
            if self.em > 1:
                sucs.append(MCState(self.wm + 2, self.wc, not self.boat))
            if self.em > 0:
                sucs.append(MCState(self.wm + 1, self.wc, not self.boat))
            if self.ec > 1:
                sucs.append(MCState(self.wm, self.wc + 2, not self.boat))
            if self.ec > 0:
                sucs.append(MCState(self.wm, self.wc + 1, not self.boat))
            if (self.ec > 0) and (self.em > 0):
                sucs.append(MCState(self.wm + 1, self.wc + 1, not self.boat))
        return [x for x in sucs if x.is_legal]

In [98]:
crossing_prob = MCState(missionaries=3, cannibals=3, boat=True)
print(crossing_prob)

On the west bank there are 3 missionaries and 3 cannibals.
On the east bank there are 0 missionaries and 0 cannibals.
The boat is on the west bank.


In [99]:
for state in crossing_prob.successors():
    print(state)
    print('\n')

On the west bank there are 3 missionaries and 1 cannibals.
On the east bank there are 0 missionaries and 2 cannibals.
The boat is on the east bank.


On the west bank there are 3 missionaries and 2 cannibals.
On the east bank there are 0 missionaries and 1 cannibals.
The boat is on the east bank.


On the west bank there are 2 missionaries and 2 cannibals.
On the east bank there are 1 missionaries and 1 cannibals.
The boat is on the east bank.




In [88]:
def display_solution(path: List[MCState]):
    if len(path) == 0: # sanity check
        return
    old_state: MCState = path[0]
    print(old_state)
    for current_state in path[1:]:
        if current_state.boat:
            print("{} missionaries and {} cannibals moved from the east bank to the west bank.\n"
                  .format(old_state.em - current_state.em, old_state.ec -
     current_state.ec))
        else:
            print("{} missionaries and {} cannibals moved from the west bank to the east bank.\n"
                  .format(old_state.wm - current_state.wm, old_state.wc -
     current_state.wc))
        print(current_state)
        old_state = current_state

In [89]:
end = bfs(crossing_prob, goal_test=MCState.goal_test, successors=MCState.successors)
if end is not None:
    print(display_solution(node_to_path(end)))

Found a solution exploring 11332 states.
On the west bank there are 3 missionaries and 3 cannibals.
On the east bank there are 0 missionaries and 0 cannibals.
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west bank to the east bank.

On the west bank there are 3 missionaries and 1 cannibals.
On the east bank there are 0 missionaries and 2 cannibals.
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east bank to the west bank.

On the west bank there are 3 missionaries and 2 cannibals.
On the east bank there are 0 missionaries and 1 cannibals.
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west bank to the east bank.

On the west bank there are 3 missionaries and 0 cannibals.
On the east bank there are 0 missionaries and 3 cannibals.
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east bank to the west bank.

On the west bank there are 3 missionaries and 1 cannibals.
On the e

In [90]:
no_solution = MCState(missionaries=1, cannibals=3, boat=True)
end = bfs(no_solution, goal_test=MCState.goal_test, successors=MCState.successors)
if end is not None:
    print(display_solution(node_to_path(end)))

Found a solution exploring 60 states.
On the west bank there are 1 missionaries and 3 cannibals.
On the east bank there are 2 missionaries and 0 cannibals.
The boat is on the west bank.
1 missionaries and 1 cannibals moved from the west bank to the east bank.

On the west bank there are 0 missionaries and 2 cannibals.
On the east bank there are 3 missionaries and 1 cannibals.
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east bank to the west bank.

On the west bank there are 0 missionaries and 3 cannibals.
On the east bank there are 3 missionaries and 0 cannibals.
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west bank to the east bank.

On the west bank there are 0 missionaries and 1 cannibals.
On the east bank there are 3 missionaries and 2 cannibals.
The boat is on the east bank.
1 missionaries and 0 cannibals moved from the east bank to the west bank.

On the west bank there are 1 missionaries and 1 cannibals.
On the east

In [91]:
if __name__ == "__main__":
    start: MCState = MCState(MAX_NUM, MAX_NUM, True)
    solution: Optional[Node[MCState]] = bfs(start, MCState.goal_test,
     MCState.successors)
    if solution is None:
        print("No solution found!")
    else:
        path: List[MCState] = node_to_path(solution)
        display_solution(path)

Found a solution exploring 11332 states.
On the west bank there are 3 missionaries and 3 cannibals.
On the east bank there are 0 missionaries and 0 cannibals.
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west bank to the east bank.

On the west bank there are 3 missionaries and 1 cannibals.
On the east bank there are 0 missionaries and 2 cannibals.
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east bank to the west bank.

On the west bank there are 3 missionaries and 2 cannibals.
On the east bank there are 0 missionaries and 1 cannibals.
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west bank to the east bank.

On the west bank there are 3 missionaries and 0 cannibals.
On the east bank there are 0 missionaries and 3 cannibals.
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east bank to the west bank.

On the west bank there are 3 missionaries and 1 cannibals.
On the e

# Exercises: Solved with Knowledge

## 1. Binary versus Linear Search

### Compare on 1_000_000 Numbers

In [92]:
import numpy as np
long_list = np.random.choice(np.arange(11), size=1_000_000)
len(long_list)

1000000

In [93]:
def linear_contains(key, search_list):
    for x in search_list:
        if x == key:
            return True
    return False

In [94]:
%%timeit -r 3 -n 100
linear_contains(11, long_list)

KeyboardInterrupt: 

In [ ]:
def binary_contains(key, search_list):
    low = 0
    high = len(search_list) - 1
    
    while low <= high:
        mid = (low + high) // 2
        
        if search_list[mid] < key:
            low = mid + 1
            
        elif search_list[mid] > key:
            high = mid - 1
        
        else:
            return True
        
    return False 

In [ ]:
sorted_long_list = sorted(long_list)

In [ ]:
%%timeit -r 3 -n 100
binary_contains(11, sorted_long_list)

In [ ]:
%%timeit -r 3 -n 100
binary_contains(11, long_list)

In [ ]:
%%timeit -r 3 -n 100
11 in long_list

In [ ]:
long_list_contains = np.random.choice(np.arange(10000), size=1_000_000)
len(long_list_contains)

In [ ]:
%%timeit -r 3 -n 100
500 in long_list_contains

In [ ]:
sorted_long_list_contains = sorted(long_list_contains)

In [ ]:
%%timeit -r 3 -n 100
binary_contains(500, sorted_long_list_contains)

In [ ]:
%%timeit -r 3 -n 100
linear_contains(500, long_list_contains)

## Add Counter to dfs, bfs, and astar

### Look at Different Counts for Same Maze

In [109]:
# generic_search.py
# From Classic Computer Science Problems in Python Chapter 2
# Copyright 2018 David Kopec
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
from __future__ import annotations
from typing import TypeVar, Iterable, Sequence, Generic, List, Callable, Set, Deque, Dict, Any, Optional
from typing_extensions import Protocol
from heapq import heappush, heappop

T = TypeVar('T')


def linear_contains(iterable: Iterable[T], key: T) -> bool:
    for item in iterable:
        if item == key:
            return True
    return False


C = TypeVar("C", bound="Comparable")


class Comparable(Protocol):
    def __eq__(self, other: Any) -> bool:
        ...

    def __lt__(self: C, other: C) -> bool:
        ...

    def __gt__(self: C, other: C) -> bool:
        return (not self < other) and self != other

    def __le__(self: C, other: C) -> bool:
        return self < other or self == other

    def __ge__(self: C, other: C) -> bool:
        return not self < other


def binary_contains(sequence: Sequence[C], key: C) -> bool:
    low: int = 0
    high: int = len(sequence) - 1
    while low <= high:  # while there is still a search space
        mid: int = (low + high) // 2
        if sequence[mid] < key:
            low = mid + 1
        elif sequence[mid] > key:
            high = mid - 1
        else:
            return True
    return False


class Stack(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []

    @property
    def empty(self) -> bool:
        return not self._container  # not is true for empty container

    def push(self, item: T) -> None:
        self._container.append(item)

    def pop(self) -> T:
        return self._container.pop()  # LIFO

    def __repr__(self) -> str:
        return repr(self._container)


class Node(Generic[T]):
    def __init__(self, state: T, parent: Optional[Node], cost: float = 0.0, heuristic: float = 0.0) -> None:
        self.state: T = state
        self.parent: Optional[Node] = parent
        self.cost: float = cost
        self.heuristic: float = heuristic

    def __lt__(self, other: Node) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)


def dfs(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]]) -> Optional[Node[T]]:
    # frontier is where we've yet to go
    frontier: Stack[Node[T]] = Stack()
    frontier.push(Node(initial, None))
    # explored is where we've been
    explored: Set[T] = {initial}

    state_count: int = 0
    # keep going while there is more to explore
    while not frontier.empty:
        state_count += 1
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if we found the goal, we're done
        if goal_test(current_state):
            print(f'Found a solution with depth-first search exploring {state_count} states.')
            return current_node
        # check where we can go next and haven't explored
        for child in successors(current_state):
            if child in explored:  # skip children we already explored
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None  # went through everything and never found goal


def node_to_path(node: Node[T]) -> List[T]:
    path: List[T] = [node.state]
    # work backwards from end to front
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    path.reverse()
    return path


class Queue(Generic[T]):
    def __init__(self) -> None:
        self._container: Deque[T] = Deque()

    @property
    def empty(self) -> bool:
        return not self._container  # not is true for empty container

    def push(self, item: T) -> None:
        self._container.append(item)

    def pop(self) -> T:
        return self._container.popleft()  # FIFO

    def __repr__(self) -> str:
        return repr(self._container)



class PriorityQueue(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []

    @property
    def empty(self) -> bool:
        return not self._container  # not is true for empty container

    def push(self, item: T) -> None:
        heappush(self._container, item)  # in by priority

    def pop(self) -> T:
        return heappop(self._container)  # out by priority

    def __repr__(self) -> str:
        return repr(self._container)


def astar(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]], heuristic: Callable[[T], float]) -> Optional[Node[T]]:
    # frontier is where we've yet to go
    frontier: PriorityQueue[Node[T]] = PriorityQueue()
    frontier.push(Node(initial, None, 0.0, heuristic(initial)))
    # explored is where we've been
    explored: Dict[T, float] = {initial: 0.0}

    state_count: int = 0
    # keep going while there is more to explore
    while not frontier.empty:
        state_count += 1
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if we found the goal, we're done
        if goal_test(current_state):
            print(f'Found a solution using astar exploring {state_count} states.')
            return current_node
        # check where we can go next and haven't explored
        for child in successors(current_state):
            new_cost: float = current_node.cost + 1  # 1 assumes a grid, need a cost function for more sophisticated apps

            if child not in explored or explored[child] > new_cost:
                explored[child] = new_cost
                frontier.push(Node(child, current_node, new_cost, heuristic(child)))
    return None  # went through everything and never found goal


if __name__ == "__main__":
    print(linear_contains([1, 5, 15, 15, 15, 15, 20], 5))  # True
    print(binary_contains(["a", "d", "e", "f", "z"], "f"))  # True
    print(binary_contains(["john", "mark", "ronald", "sarah"], "sheila"))  # False

True
True
False


In [ ]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random
from math import sqrt


class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    START = "S"
    GOAL = "G"
    PATH = "*"


class MazeLocation(NamedTuple):
    row: int
    column: int


class Maze:
    def __init__(self, rows: int = 10, columns: int = 10, sparseness: float = 0.2, start: MazeLocation = MazeLocation(0, 0), goal: MazeLocation = MazeLocation(9, 9)) -> None:
        # initialize basic instance variables
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
        # fill the grid with empty cells
        self._grid: List[List[Cell]] = [[Cell.EMPTY for c in range(columns)] for r in range(rows)]
        # populate the grid with blocked cells
        self._randomly_fill(rows, columns, sparseness)
        # fill the start and goal locations in
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL

    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED

    # return a nicely formatted version of the maze for printing
    def __str__(self) -> str:
        output: str = ""
        for row in self._grid:
            output += "|".join([c.value for c in row]) + "\n"
        return output

    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal

    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations

    def mark(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
    
    def clear(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL


def euclidean_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = ml.column - goal.column
        ydist: int = ml.row - goal.row
        return sqrt((xdist * xdist) + (ydist * ydist))
    return distance


def manhattan_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = abs(ml.column - goal.column)
        ydist: int = abs(ml.row - goal.row)
        return (xdist + ydist)
    return distance


if __name__ == "__main__":
    # Test DFS
    m: Maze = Maze()
    print(m)
    solution1: Optional[Node[MazeLocation]] = dfs(m.start, m.goal_test, m.successors)
    if solution1 is None:
        print("No solution found using depth-first search!")
    else:
        path1: List[MazeLocation] = node_to_path(solution1)
        m.mark(path1)
        print(m)
        m.clear(path1)
    # Test BFS
    solution2: Optional[Node[MazeLocation]] = bfs(m.start, m.goal_test, m.successors)
    if solution2 is None:
        print("No solution found using breadth-first search!")
    else:
        path2: List[MazeLocation] = node_to_path(solution2)
        m.mark(path2)
        print(m)
        m.clear(path2)
    # Test A*
    distance: Callable[[MazeLocation], float] = manhattan_distance(m.goal)
    solution3: Optional[Node[MazeLocation]] = astar(m.start, m.goal_test, m.successors, distance)
    if solution3 is None:
        print("No solution found using A*!")
    else:
        path3: List[MazeLocation] = node_to_path(solution3)
        m.mark(path3)
        print(m)

In [ ]:
my_maze = Maze(rows=20, columns=20, start=MazeLocation(2, 2),
               goal=MazeLocation(16, 16))
print(my_maze)

In [110]:
def show_solution(my_maze, method):
    """
    Solve a maze with the given method (either bfs or dfs).
    """
    end = method(my_maze.start, 
                 my_maze.goal_test, 
                 my_maze.successors)
    if end is None:
        print('No solution was found!')
        return
    path = node_to_path(end)
    my_maze.mark(path)
    print(my_maze)
    my_maze.clear(path)

In [111]:
test_maze = Maze(rows=40, columns=40, goal=MazeLocation(39, 39))

In [112]:
show_solution(test_maze, bfs)

Found a solution with breadth first search exploring 1285 states.
S| | | | | | | | | | |X| |X| | | | | | | | | | | |X| | | | |X| | | | | | | | |X
*| | |X| | | |X|X|X| | | | |X| | | | |X| | | | | | | | | |X| | | | | | | | | | 
*| | | | | | | |X| | | | | |X|X| | |X| | | | | | | |X| | | | |X| | |X| |X| | | 
*| | | | |X| | | | | | | | |X| | | | | | | | | | |X| | | | | | | | | |X| | | |X
*|*|*| | |X|X| |X| | | | | | | | | |X| | | | | | | | | | |X| | | | | | | | | | 
X|X|*|X| | | | | | |X| | |X|X| | |X| | | | | | | | | | |X| | | | |X| | | | | | 
 | |*|X| | | | | | |X| | |X| |X| | |X|X| | | | | | |X| | | | | |X| | |X| | | | 
 | |*|X| |X|X| | | | | | | | | | | | | | | | |X| | | | | | | | | | | | |X|X| | 
 | |*| | | |X| | | | | | | | |X| |X| | | | | | | | | | | |X| | | |X| | | | |X| 
X|X|*| | | | | | | |X| | |X|X| | | | |X|X| | |X| | | | | | | | | | | | | | | | 
X|X|*| | |X|X|X|X| | | |X| | |X| | | | | | |X|X|X| | | | |X|X| | | |X|X| | |X| 
 | |*| | |X|X| | | | | |X| | | | | | | | | | |X| | |X|

In [113]:
show_solution(test_maze, dfs)

Found a solution with depth-first search exploring 898 states.
S|*|*|*|*|*|*|*|*|*|*|X| |X| | | | | | | | | | | |X| | | | |X| | | | | | | | |X
 | | |X| | | |X|X|X|*|*|*|*|X| | | | |X| | | | | | | | | |X| | | | | | | | | | 
*|*|*|*|*|*|*| |X| | | | |*|X|X| | |X| | | | | | | |X| | | | |X| | |X| |X| | | 
*| | | | |X|*|*|*|*|*|*|*|*|X| | | | | | | | | | |X| | | | | | | | | |X| | | |X
*|*|*|*|*|X|X| |X| | | | | | | | | |X| | | | | | | | | | |X| | | | | | | | | | 
X|X| |X|*|*|*|*|*|*|X| | |X|X| | |X| | | | | | | | | | |X| | | | |X| | | | | | 
 | | |X| | | | | |*|X| | |X| |X| | |X|X| | | | | | |X| | | | | |X| | |X| | | | 
 | | |X| |X|X|*|*|*| | | | | | | | | | | | | |X| | | | | | | | | | | | |X|X| | 
 | | | | | |X|*| | | | | | | |X| |X| | | | | | | | | | | |X| | | |X| | | | |X| 
X|X|*|*|*|*|*|*| | |X| | |X|X| | | | |X|X| | |X| | | | | | | | | | | | | | | | 
X|X|*| | |X|X|X|X| | | |X| | |X| | | | | | |X|X|X| | | | |X|X| | | |X|X| | |X| 
*|*|*| | |X|X| | | | | |X| | | | | | | | | | |X| | |X| | 

In [116]:
def show_solution_astar(my_maze, method, distance):
    """
    Solve a maze with the given method and distance.
    """
    end = method(my_maze.start, 
                 my_maze.goal_test, 
                 my_maze.successors,
                distance)
    if end is None:
        print('No solution was found!')
        return
    path = node_to_path(end)
    my_maze.mark(path)
    print(my_maze)
    my_maze.clear(path)

In [117]:
distance = manhattan_distance(test_maze.goal)
distance

<function __main__.manhattan_distance.<locals>.distance(ml: 'MazeLocation') -> 'float'>

In [123]:
show_solution_astar(test_maze, astar, distance)

Found a solution using astar exploring 576 states.
S| | | | | | | | | | |X| |X| | | | | | | | | | | |X| | | | |X| | | | | | | | |X
*| | |X| | | |X|X|X| | | | |X| | | | |X| | | | | | | | | |X| | | | | | | | | | 
*| | | | | | | |X| | | | | |X|X| | |X| | | | | | | |X| | | | |X| | |X| |X| | | 
*|*|*| | |X| | | | | | | | |X| | | | | | | | | | |X| | | | | | | | | |X| | | |X
 | |*| | |X|X| |X| | | | | | | | | |X| | | | | | | | | | |X| | | | | | | | | | 
X|X|*|X| | | | | | |X| | |X|X| | |X| | | | | | | | | | |X| | | | |X| | | | | | 
 | |*|X| | | | | | |X| | |X| |X| | |X|X| | | | | | |X| | | | | |X| | |X| | | | 
 | |*|X| |X|X| | | | | | | | | | | | | | | | |X| | | | | | | | | | | | |X|X| | 
 | |*| | | |X| | | | | | | | |X| |X| | | | | | | | | | | |X| | | |X| | | | |X| 
X|X|*| | | | | | | |X| | |X|X| | | | |X|X| | |X| | | | | | | | | | | | | | | | 
X|X|*| | |X|X|X|X| | | |X| | |X| | | | | | |X|X|X| | | | |X|X| | | |X|X| | |X| 
 | |*| | |X|X| | | | | |X| | | | | | | | | | |X| | |X| | | |X| | | | 

In [124]:
distance = euclidean_distance(test_maze.goal)
distance

<function __main__.euclidean_distance.<locals>.distance(ml: 'MazeLocation') -> 'float'>

In [125]:
show_solution_astar(test_maze, astar, distance)

Found a solution using astar exploring 1135 states.
S| | | | | | | | | | |X| |X| | | | | | | | | | | |X| | | | |X| | | | | | | | |X
*|*| |X| | | |X|X|X| | | | |X| | | | |X| | | | | | | | | |X| | | | | | | | | | 
 |*|*|*| | | | |X| | | | | |X|X| | |X| | | | | | | |X| | | | |X| | |X| |X| | | 
 | | |*| |X| | | | | | | | |X| | | | | | | | | | |X| | | | | | | | | |X| | | |X
 | | |*|*|X|X| |X| | | | | | | | | |X| | | | | | | | | | |X| | | | | | | | | | 
X|X| |X|*|*| | | | |X| | |X|X| | |X| | | | | | | | | | |X| | | | |X| | | | | | 
 | | |X| |*|*|*| | |X| | |X| |X| | |X|X| | | | | | |X| | | | | |X| | |X| | | | 
 | | |X| |X|X|*|*| | | | | | | | | | | | | | |X| | | | | | | | | | | | |X|X| | 
 | | | | | |X| |*| | | | | | |X| |X| | | | | | | | | | | |X| | | |X| | | | |X| 
X|X| | | | | | |*|*|X| | |X|X| | | | |X|X| | |X| | | | | | | | | | | | | | | | 
X|X| | | |X|X|X|X|*|*|*|X| | |X| | | | | | |X|X|X| | | | |X|X| | | |X|X| | |X| 
 | | | | |X|X| | | | |*|X| | | | | | | | | | |X| | |X| | | |X| | | |